# Strong Lens in ECDFS location

- author Sylvie Dagoret-Campagne
- creation date 2025-06-02
- last update 2025-06-08

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay

from lsst.skymap import PatchInfo, Index2D

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u

In [ ]:
#%matplotlib widget

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "xx-large"
plt.rcParams["axes.titlesize"] = "xx-large"
plt.rcParams["xtick.labelsize"] = "xx-large"
plt.rcParams["ytick.labelsize"] = "xx-large"

In [ ]:
def plot_tract_patches_ra_dec(butler, skymap, tract_id, df_points = None):
    
    tract_info = skymap[tract_id]
    wcs = tract_info.getWcs()

    # Dimensions des patches
    num_patches_x, num_patches_y = tract_info.getNumPatches()

    fig, ax = plt.subplots(figsize=(10, 8))

    for y in range(num_patches_y):
        for x in range(num_patches_x):
            patch_info = tract_info.getPatchInfo((x, y))
            patch_bbox = patch_info.getOuterBBox()

            # Coins en pixels
            corners = [
                (patch_bbox.getMinX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMaxY()),
                (patch_bbox.getMinX(), patch_bbox.getMaxY()),
            ]

            # Conversion pixel -> RA, Dec (pas besoin de .toIcrs())
            sky_coords = [wcs.pixelToSky(xp, yp) for xp, yp in corners]
            ra = [coord.getRa().asDegrees() for coord in sky_coords]
            dec = [coord.getDec().asDegrees() for coord in sky_coords]

            # Ferme le polygone
            ra.append(ra[0])
            dec.append(dec[0])

            ax.plot(ra, dec, color="blue", linewidth=0.5)

    # Ajout des points (optionnel)
    if df_points is not None:
        df = df_points[df_points["tract"] == tract_id]  # filtre les points du tract courant
        ax.scatter(df["ra"], df["dec"], color="red", marker="o", s=30, label="Sources", zorder=5)

        # Optionnel : afficher les noms (ex: df["field_name"])
        for _, row in df.iterrows():
            ax.text(row["ra"], row["dec"], row["field_name"], fontsize=7, ha='right', va='bottom', color='darkred')

    ax.set_xlabel("RA [deg]")
    ax.set_ylabel("Dec [deg]")
    ax.set_aspect('equal', adjustable='datalim')
    ax.set_title(f"Patches in Tract {tract_id} ")
    ax.invert_xaxis()
    ax.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_tract_patches_ra_dec(butler, skymap, tract_id, df_points=None):
    # Ouvre le Butler
    
    tract_info = skymap[tract_id]
    wcs = tract_info.getWcs()
    num_patches_x, num_patches_y = tract_info.getNumPatches()

    fig, ax = plt.subplots(figsize=(12, 10))

    for y in range(num_patches_y):
        for x in range(num_patches_x):
            patch_info = tract_info.getPatchInfo((x, y))
            patch_seqnum = patch_info.getSequentialIndex()
            patch_bbox = patch_info.getOuterBBox()

            # Coins du patch
            corners = [
                (patch_bbox.getMinX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMinY()),
                (patch_bbox.getMaxX(), patch_bbox.getMaxY()),
                (patch_bbox.getMinX(), patch_bbox.getMaxY()),
            ]
            sky_coords = [wcs.pixelToSky(xp, yp) for xp, yp in corners]
            ra = [coord.getRa().asDegrees() for coord in sky_coords]
            dec = [coord.getDec().asDegrees() for coord in sky_coords]
            ra.append(ra[0])
            dec.append(dec[0])
            ax.plot(ra, dec, color="blue", linewidth=0.5)

            # Centre du patch (en pixels)
            center_pixel_x = 0.5 * (patch_bbox.getMinX() + patch_bbox.getMaxX())
            center_pixel_y = 0.5 * (patch_bbox.getMinY() + patch_bbox.getMaxY())
            center_sky = wcs.pixelToSky(center_pixel_x, center_pixel_y)
            center_ra = center_sky.getRa().asDegrees()
            center_dec = center_sky.getDec().asDegrees()

            # Numéro de patch (x, y)
            #ax.text(center_ra, center_dec, f"{x},{y}", fontsize=6, ha='center', va='center', color='black')
            ax.text(center_ra, center_dec, f"{patch_seqnum}", fontsize=10, ha='center', va='center', color='darkgreen')

    # Ajout des points (optionnel)
    if df_points is not None:
        df = df_points[df_points["tract"] == tract_id]
        ax.scatter(df["ra"], df["dec"], color="red", marker="o", s=30, label="Sources", zorder=5)

        # Optionnel : noms des sources
        for _, row in df.iterrows():
            ax.text(row["ra"], row["dec"], row["field_name"], fontsize=10, fontweight="bold" ,ha='right', va='bottom', color='darkred')

    ax.set_xlabel("RA [deg]")
    ax.set_ylabel("Dec [deg]")
    ax.set_title(f"Patches in Tract {tract_id}")
    ax.invert_xaxis()
    ax.set_aspect('equal', adjustable='datalim')
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    plt.show()


## Config

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = "/repo/main"
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
# collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = '" + instrument + "'"
collectionStr = collection.replace("/", "_")
BANDSEL = "r"  # Most fields were observed in red filter

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection)

In [ ]:
camera = butler.get("camera", collections=collection, instrument=instrument)

## List of Strong Lenses
- article : https://arxiv.org/pdf/1104.0931

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
ra = Angle(ra10)
print(ra.degree)
dec = Angle(dec10)
print(dec.degree)

## List of Sky field of interest

In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
df

### Make list of keys

In [ ]:
list_of_keys = sorted(list(lsstcomcam_targets.keys()))

In [ ]:
list_of_keys = ["ECDFS_G15422", "ECDFS_G34244","ECDFS_G40173","ECDFS_G43242","ECDFS_G46446",
                "ECDFS_G12589","ECDFS_G43797","ECDFS_G28294","ECDFS_G6857","ECDFS_G36714"]

In [ ]:
all_tracts = []
all_patches = []
for key in list_of_keys:
    the_target = lsstcomcam_targets[key]
    target_ra = the_target["ra"]
    target_dec = the_target["dec"]
    target_name = the_target["field_name"]
    target_point = SpherePoint(target_ra, target_dec, degrees)

    tract_info = skymap.findTract(target_point)
    patch_info = tract_info.findPatch(target_point)
    #bbox = patch_info.getOuterBBox()

    tractNbSel = tract_info.getId()
    patchNbSel = patch_info.getSequentialIndex()

    all_tracts.append(tractNbSel)
    all_patches.append(patchNbSel)

In [ ]:
df["tract"] = all_tracts
df["patch"] = all_patches

In [ ]:
df

In [ ]:
tract_id = 5063

In [ ]:
plot_tract_patches_ra_dec(butler, skymap, tract_id, df)

## Dump info on tract

In [ ]:
tract_info = skymap[tract_id]
wcs = tract_info.getWcs()
num_patches_x, num_patches_y = tract_info.getNumPatches()

In [ ]:
tract_info.getBBox()

In [ ]:
tract_info.getOuterSkyPolygon()

In [ ]:
wcs

In [ ]:
for y in range(num_patches_y):
    for x in range(num_patches_x):
        patch_info = tract_info.getPatchInfo((x, y))
        patch_seqnum = patch_info.getSequentialIndex()
        patch_bbox = patch_info.getOuterBBox()
        print(f"x={x} , y={y}, seq = {patch_seqnum}", patch_info)